# Install Essential Package

In [2]:


!pip install  tqdm
!pip install ultralytics
!pip install torch torchvision torchaudio
!pip install utils

import torch
import utils
from IPython.display import Image  # for displaying images
import os
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw ,ImageFont
import numpy as np
import matplotlib.pyplot as plt
import cv2


font_path = os.path.join(cv2.__path__[0],'qt','fonts','DejaVuSans.ttf')
random.seed(108)


# Show Label On Image
## Extract Info From Xml
## change PASCAL VOC .xml to yolov5 .txt

In [3]:
!pwd

/home/ray/StomataYolov8


In [7]:
# Function to get the data from XML Annotation
def extract_info_from_xml(xml_file):
    root = ET.parse(xml_file).getroot()

    # Initialise the info dict
    info_dict = {}
    info_dict['bboxes'] = []
    
    for elem in root:
        # Get the file name
        if elem.tag == "filename":
            info_dict['filename'] = elem.text

        # Get the image size
        elif elem.tag == "size":
            image_size = []
            for subelem in elem:
                image_size.append(int(subelem.text))

            info_dict['image_size'] = tuple(image_size)

        # Get details of the bounding box
        elif elem.tag == "object":
            bbox = {}
            for subelem in elem:
                if subelem.tag == "name":
                    bbox["class"] = subelem.text

                elif subelem.tag == "robndbox":
                    for subsubelem in subelem:
                        bbox[subsubelem.tag] = int(float(subsubelem.text))
            info_dict['bboxes'].append(bbox)

    return info_dict


#xml choose complete

class_name_to_id_mapping = {"complete": 0,
                "incomplete": 1,
                           "blurry.complete": 2,
                           "blurry.incomplete" :3 , #
                           "hair":4} # human hair

# change robndbox to yolov5 format
def convert_to_yolov8(info_dict) :
    print_buffer = []

    # For each bounding box
    for b in info_dict["bboxes"]:
        try:
            class_id = class_name_to_id_mapping[b["class"]]
        except KeyError:
            print("Invalid Class. Must be one from ", class_name_to_id_mapping.keys())

        # robndbox format
        # Transform the bbox co-ordinates as per the format required by YOLO v5
        b_center_x = int(b["cx"])
        b_center_y = int(b["cy"])
        b_width = int(b["w"])
        b_height = int(b["h"])

        # Normalise the co-ordinates by the dimensions of the image
        image_w, image_h, image_c = info_dict["image_size"]
        b_center_x /= image_w
        b_center_y /= image_h
        b_width    /= image_w
        b_height   /= image_h

        #Write the bbox details to the file
        print_buffer.append("{} {:.3f} {:.3f} {:.3f} {:.3f}".format(class_id, b_center_x, b_center_y, b_width, b_height))

    # Name of the file which we have to save
    save_file_name = os.path.join("./AllDataset", info_dict["filename"] + '.txt' )

    # Save the annotation to disk
    print("\n".join(print_buffer), file= open(save_file_name, "w"))

In [8]:
# Get the annotationse
annotations = [os.path.join('./AllData', x) for x in os.listdir('./AllData') if x[-3:] == "xml"]


annotations.sort()
#print(annotations)
# Convert and save the annotations
for ann in tqdm(annotations):
    info_dict = extract_info_from_xml(ann)
    convert_to_yolov8(info_dict)
    
annotations = [os.path.join('./AllData', x) for x in os.listdir('./AllData') if x[-3:] == "txt"]
print(annotations)

FileNotFoundError: [Errno 2] No such file or directory: './AllDataset'

# Testing the annotations

## Label the class on the image
Just for a sanity check, let us now test some of these transformed annotations. We randomly load one of the annotations and plot boxes using the transformed annotations, and visually inspect it to see whether our code has worked as intended.

Run the next cell multiple times. Every time, a random annotation is sampled.

Use command mv ./T22/*_label.jpg ./T22Label  
move label image to folder

In [7]:
random.seed(0)


class_name_to_id_mapping = {"complete": 0,
                "incomplete": 1,
                           "blurry.complete": 2,
                           "blurry.incomplete" :3 , 
                           "hair":4
                           } # human hair

class_id_to_name_mapping = dict(zip(class_name_to_id_mapping.values(), class_name_to_id_mapping.keys()))

fnt = ImageFont.truetype("Pillow/Tests/fonts/FreeMono.ttf", 40)



def PlotAndSavebounding_box(image_name,image, annotation_list):
    annotations = np.array(annotation_list)
    w, h = image.size

    plotted_image = ImageDraw.Draw(image)

    transformed_annotations = np.copy(annotations)
    transformed_annotations[:,[1,3]] = annotations[:,[1,3]] * w
    transformed_annotations[:,[2,4]] = annotations[:,[2,4]] * h

    transformed_annotations[:,1] = transformed_annotations[:,1] - (transformed_annotations[:,3] / 2)
    transformed_annotations[:,2] = transformed_annotations[:,2] - (transformed_annotations[:,4] / 2)
    transformed_annotations[:,3] = transformed_annotations[:,1] + transformed_annotations[:,3]
    transformed_annotations[:,4] = transformed_annotations[:,2] + transformed_annotations[:,4]

    for ann in transformed_annotations:
        obj_cls, x0, y0, x1, y1 = ann
        plotted_image.rectangle(((x0,y0), (x1,y1)),outline ="red", width = 3)
        plotted_image.text((x0, y0-10), class_id_to_name_mapping[(int(obj_cls))] , fill = (255,0,255,255), font = fnt)
    
    # covert PIL image to cv2 image
    open_cv_image = np.array(image)

    # Convert RGB to BGR
    open_cv_image = open_cv_image[:, :, ::-1].copy()

    print(type(image))


    image_name = image_name.replace(".jpg", "_label.jpg")
    
    cv2.imwrite( image_name , open_cv_image, [cv2.IMWRITE_JPEG_QUALITY, 100])

    #plt.imshow(np.array(image))
    #plt.show()


# Get any random annotation file
#annotation_file = annotations[0]

'''
with open(annotation_file, "r") as file:
    annotation_list = file.read().split("\n")[:-1]
    annotation_list = [x.split(" ") for x in annotation_list]
    annotation_list = [[float(y) for y in x ] for x in annotation_list]
'''    


# use %cd T22Label to locate folder

# Plot the Bounding Box
# Polt all the Images Labels and annotations
for i in range(0,len(annotations)):

    annotation_file = annotations[i]

    with open(annotation_file, "r") as file:
        annotation_list = file.read().split("\n")[:-1]
        annotation_list = [x.split(" ") for x in annotation_list]
        annotation_list = [[float(y) for y in x ] for x in annotation_list]

    #Get the corresponding image file
    image_file = annotation_file.replace("txt", "jpg")
    
    assert os.path.exists(image_file)

    #Load the image

    image = Image.open(image_file)
    PlotAndSavebounding_box(image_file,image, annotation_list)



NameError: name 'annotations' is not defined

### Count all class of images
export the csv file of the dataset

In [40]:

annotations = [os.path.join('/home/ray/datasets/T32/labels/train', x) for x in os.listdir('/home/ray/datasets/T32/labels/train') if x[-3:] == "txt"]

import pandas as pd


class_name_to_id_mapping = {"complete": 0,
                "incomplete": 1,
                           "blurry.complete": 2,
                           "blurry.incomplete" :3 , 
                           "hair":4
                           } # human hair

class_id_to_name_mapping = dict(zip(class_name_to_id_mapping.values(), class_name_to_id_mapping.keys()))




def CountAndSaveInstance(dic, annotation_list) :
    annotations = np.array(annotation_list)
    

    
    transformed_annotations = np.copy(annotations)
    
    for ann in transformed_annotations:
        obj_cls, x0, y0, x1, y1 = ann
        dic["count"][int(obj_cls)] += 1
    
    
# Set original  
dic = { "class" : class_name_to_id_mapping.keys() , "count" : [0,0,0,0,0]}



for i in range(0,len(annotations)):

    annotation_file = annotations[i]

    with open(annotation_file, "r") as file:
        annotation_list = file.read().split("\n")[:-1]
        annotation_list = [x.split(" ") for x in annotation_list]
        annotation_list = [[float(y) for y in x ] for x in annotation_list]

    #
    CountAndSaveInstance(dic,annotation_list)


In [42]:
dic_df = pd.DataFrame(dic)
dic_df

,class,count
0,complete,862
1,incomplete,62
2,blurry.complete,587
3,blurry.incomplete,120
4,hair,874


In [43]:
dic_df.to_csv('T22Count.csv')

# Read Image and Annotation
###   Split to train datasets and Test data Set

In [8]:
# Read images and annotations
images = [os.path.join('./T32', x) for x in os.listdir('./T32') if x[-3:] == 'jpg' ]
annotations = [os.path.join('./T32', x) for x in os.listdir('./T32') if x[-3:] == "txt"]

images.sort()
annotations.sort()

# Split the dataset into train-valid-test splits
train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)
val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)

## Create the folders to keep the splits.

In [9]:
!mkdir ./T32/images ./T32/labels

In [10]:
!mkdir ./T32/images/train ./T32/images/val ./T32/images/test  ./T32/labels/train  ./T32/labels/val  ./T32/labels/test


## Move the files to their respective folders

In [11]:
#Utility function to move images
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False


# Move the splits into their folders
move_files_to_folder(train_images, './T32/images/train')
move_files_to_folder(val_images, './T32/images/val/')
move_files_to_folder(test_images, './T32/images/test/')
move_files_to_folder(train_annotations, './T32/labels/train/')
move_files_to_folder(val_annotations, './T32/labels/val/')
move_files_to_folder(test_annotations, './T32/labels/test/')

##### Create a new file called road_sign_data.yaml and place it in the yolov5/data folder. Then populate it with the following.

train : ./T22/images/train/
val : ./T22/images/val/
test : ./T22/images/test/


##### number of classes
nc: 5

##### class names
names : ["complete", "incomplete","blurry.complete","blurry.incomplete" , "hair"]

# Train the Yolov8 Model

In [9]:
!pip install -U ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 699.2/699.2 kB 6.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: ultralytics
    Found existing installation: ultralytics 8.0.232
    Uninstalling ultralytics-8.0.232:
      Successfully uninstalled ultralytics-8.0.232


In [ ]:
% pip install 

In [1]:
from ultralytics import YOLO

model = YOLO("yolov8m-obb.pt")


/home/ray/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from ultralytics import YOLO


# Load a model
model = YOLO("/home/ray/runs/detect/T22+T32Train/weights/best.pt")  # load an official model
results = model.train(data="data_T22T32.yaml", epochs=250)
# Validate the model
results = model.val(data="data_T22T32.yaml")  # no arguments needed, dataset and settings remembered

New https://pypi.org/project/ultralytics/8.0.238 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.232 🚀 Python-3.10.13 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7937MiB)
engine/trainer: task=detect, mode=train, model=/home/ray/runs/detect/T22+T32Train/weights/best.pt, data=data_T22T32.yaml, epochs=250, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train7, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, 

train: Scanning /home/ray/datasets/T22+T32/labels/train.cache... 146 images, 0 backgrounds, 0 corrupt: 100%|██████████| 146/146 [00:00<?, ?it/s]
val: Scanning /home/ray/datasets/T22+T32/labels/val.cache... 17 images, 0 backgrounds, 0 corrupt: 100%|██████████| 17/17 [00:00<?, ?it/s]


Plotting labels to /home/ray/runs/detect/train7/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
250 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/250      5.04G     0.6861     0.3726     0.8243        214        640: 100%|██████████| 10/10 [00:02<00:00,  3.87it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00, 11.03it/s]

                   all         17        829      0.798      0.753      0.805      0.576


...


54 epochs completed in 0.039 hours.
Optimizer stripped from /home/ray/runs/detect/train7/weights/last.pt, 22.5MB
Optimizer stripped from /home/ray/runs/detect/train7/weights/best.pt, 22.5MB

Validating /home/ray/runs/detect/train7/weights/best.pt...
Ultralytics YOLOv8.0.232 🚀 Python-3.10.13 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7937MiB)
Model summary (fused): 168 layers, 11127519 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  9.54it/s]


                   all         17        829      0.802      0.744      0.806      0.596
              complete         17        212      0.794      0.856      0.831       0.69
            incomplete         17         32      0.781      0.688      0.749      0.577
       blurry.complete         17        198      0.844      0.773      0.807      0.646
     blurry.incomplete         17         24      0.669      0.667      0.765      0.573
                  hair         17        363      0.921      0.736      0.879      0.494
Speed: 0.1ms preprocess, 1.9ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to /home/ray/runs/detect/train7
Ultralytics YOLOv8.0.232 🚀 Python-3.10.13 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7937MiB)
Model summary (fused): 168 layers, 11127519 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning /home/ray/datasets/T22+T32/labels/val.cache... 17 images, 0 backgrounds, 0 corrupt: 100%|██████████| 17/17 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.00it/s]


                   all         17        829      0.801      0.743      0.809      0.597
              complete         17        212      0.794      0.855      0.831      0.694
            incomplete         17         32      0.781      0.688       0.77       0.57
       blurry.complete         17        198      0.843      0.773      0.806      0.645
     blurry.incomplete         17         24      0.669      0.667      0.765      0.581
                  hair         17        363      0.917      0.733      0.873      0.495
Speed: 0.2ms preprocess, 4.1ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to /home/ray/runs/detect/train73


In [4]:
model = YOLO("/home/ray/runs/detect/T22+T32Train/weights/best.pt")  # load an official model 
results = model.val(data="data_T22T32.yaml")  # no arguments needed, dataset and settings remembered

Ultralytics YOLOv8.0.232 🚀 Python-3.10.13 torch-2.1.0+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7937MiB)


Model summary (fused): 168 layers, 11127519 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning /home/ray/datasets/T22+T32/labels/val.cache... 17 images, 0 backgrounds, 0 corrupt: 100%|██████████| 17/17 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  2.02it/s]


                   all         17        829      0.775      0.792       0.82      0.603
              complete         17        212      0.768      0.882      0.801      0.687
            incomplete         17         32      0.753      0.688      0.835      0.614
       blurry.complete         17        198      0.811      0.773      0.774      0.643
     blurry.incomplete         17         24      0.674      0.792      0.794      0.561
                  hair         17        363       0.87      0.826      0.895       0.51
Speed: 0.2ms preprocess, 7.6ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to /home/ray/runs/detect/val4


### Predict

In [7]:
from ultralytics import YOLO

model = YOLO("/home/ray/runs/detect/T22+T32Train/weights/best.pt")  # load a custom model

# Predict with the model
results = model.predict( '/home/ray/Downloads/Stomata/T22+T32',save=True,save_crop = False,save_txt = True,save_conf = True)  # predict on an image


image 1/166 /home/ray/Downloads/Stomata/T22+T32/32TL600_W1_GC1_R1_P14_G35_4.jpg: 480x640 26 completes, 10 blurry.completes, 1 blurry.incomplete, 48 hairs, 79.5ms
image 2/166 /home/ray/Downloads/Stomata/T22+T32/32TL600_W1_GC1_R1_P14_G35_5.jpg: 480x640 34 completes, 1 incomplete, 5 blurry.completes, 20 hairs, 3.5ms
image 3/166 /home/ray/Downloads/Stomata/T22+T32/32TL600_W1_GC1_R1_P18_G13_4.jpg: 480x640 21 completes, 18 blurry.completes, 66 hairs, 4.1ms
image 4/166 /home/ray/Downloads/Stomata/T22+T32/32TL600_W1_GC1_R1_P18_G13_7.jpg: 480x640 25 completes, 19 blurry.completes, 2 blurry.incompletes, 29 hairs, 4.0ms
image 5/166 /home/ray/Downloads/Stomata/T22+T32/32TL600_W1_GC1_R1_P19_G49_3.jpg: 480x640 15 completes, 11 blurry.completes, 31 hairs, 3.8ms
image 6/166 /home/ray/Downloads/Stomata/T22+T32/32TL600_W1_GC1_R1_P19_G49_6.jpg: 480x640 28 completes, 2 incompletes, 2 blurry.completes, 7 hairs, 3.4ms
image 7/166 /home/ray/Downloads/Stomata/T22+T32/32TL600_W1_GC1_R1_P21_G54_1.jpg: 480x640 

##### Analysis Yolov8n  and Groundtruth predict  

Compare all classes count

In [10]:
import pandas as pd
import os
import numpy as np

class_name_to_id_mapping = {"complete": 0,
                "incomplete": 1,
                           "blurry.complete": 2,
                           "blurry.incomplete" :3 , 
                           "hair":4
                           } # human hair

class_id_to_name_mapping = dict(zip(class_name_to_id_mapping.values(), class_name_to_id_mapping.keys()))

#get predicts data
predicts = [os.path.join('/home/ray/runs/detect/predict2/labels', x) for x in os.listdir('/home/ray/runs/detect/predict2/labels') if x[-3:] == 'txt' ]


def GetPredictResults(predicts,dic):

    
    for i in range(0,len(predicts)):
        predict_file = predicts[i]
        with open(predict_file, "r") as file:
                predict_list = file.read().split("\n")[:-1]
                predict_list = [x.split(" ") for x in predict_list]
                predict_list = [[float(y) for y in x ] for x in predict_list]

        
        predict_list = np.array(predict_list)
    

        for ann in predict_list:
            obj_cls, x0, y0, x1, y1 , conf = ann
            dic["count"][int(obj_cls)] += 1
        
        

dic = { "class" : class_name_to_id_mapping.keys() , "count" : [0,0,0,0,0]}

GetPredictResults(predicts,dic)

dic_df = pd.DataFrame(dic)

dic_df.to_csv('T32PredictResults.csv')

dic_df



,class,count
0,complete,2044
1,incomplete,7
2,blurry.complete,1173
3,blurry.incomplete,19
4,hair,2809
